##  1_Preprocessing.ipynb – Dataset Cleaning & Sampling

This notebook prepares the Lending Club dataset for modeling.  
It filters relevant rows and columns, creates the combined BERT-compatible text field, and generates a balanced sample for efficient training.

**Main outputs:**
- `loan_data_cleaned.csv`: Cleaned full dataset (2015–2016 loans)
- `loan_data_sampled.csv`: Balanced subset (10,000 fully paid + 10,000 charged off)

### Step 1: Load the Lending Club Dataset

First, we mount Google Drive and load the full Lending Club dataset from the provided CSV.  
Since it’s a large file (~2.2M rows and 151 columns), we’ll take a quick look at its shape and structure before moving ahead.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/LoanApprovalProject/Data/accepted_2007_to_2018Q4.csv'

import pandas as pd

df_raw = pd.read_csv(data_path, low_memory=False)

print("Dataset shape:", df_raw.shape)
print("\nColumn count:", len(df_raw.columns))

df_raw.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset shape: (2260701, 151)

Column count: 151


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


### Step 2: Filter and Select Relevant Columns

We don’t need all rows or columns.  
Here, we keep only loans from 2015 and 2016 and select just the relevant features (numerical, categorical, and text) for our models.  
We also convert the target column `loan_status` into binary:  
`Fully Paid → 1`, `Charged Off → 0`.

In [3]:
df = df_raw[df_raw['issue_d'].astype(str).str.contains('2015|2016', na=False)]

# Columns I want to keep for modeling
selected_columns = [
    'loan_amnt', 'term', 'int_rate', 'grade', 'emp_title', 'home_ownership',
    'annual_inc', 'verification_status', 'purpose', 'title', 'dti',
    'open_acc', 'revol_util', 'fico_range_high', 'loan_status'
]

df = df[selected_columns].copy()

# Drop rows missing the target or key numerical values
df = df.dropna(subset=['loan_status', 'annual_inc'])

# Convert loan_status to binary: Fully Paid → 1, Charged Off → 0
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
df['loan_status'] = df['loan_status'].map({'Fully Paid': 1, 'Charged Off': 0})

print("Shape after filtering and column selection:", df.shape)
df.head()


Shape after filtering and column selection: (668640, 15)


,loan_amnt,term,int_rate,grade,emp_title,home_ownership,annual_inc,verification_status,purpose,title,dti,open_acc,revol_util,fico_range_high,loan_status
0,3600.0,36 months,13.99,C,leadman,MORTGAGE,55000.0,Not Verified,debt_consolidation,Debt consolidation,5.91,7.0,29.7,679.0,1
1,24700.0,36 months,11.99,C,Engineer,MORTGAGE,65000.0,Not Verified,small_business,Business,16.06,22.0,19.2,719.0,1
2,20000.0,60 months,10.78,B,truck driver,MORTGAGE,63000.0,Not Verified,home_improvement,NaN,10.78,6.0,56.2,699.0,1
4,10400.0,60 months,22.45,F,Contract Specialist,MORTGAGE,104433.0,Source Verified,major_purchase,Major purchase,25.37,12.0,64.5,699.0,1
5,11950.0,36 months,13.44,C,Veterinary Tecnician,RENT,34000.0,Source Verified,debt_consolidation,Debt consolidation,10.20,5.0,68.4,694.0,1


# Step 3: Create the Combined Text Field for BERT

Since I’ll be using BERT for the text input, I’m creating a new column that combines the `purpose` and `title` fields into one. I’ll fill missing titles with “Unknown” to avoid issues later.


In [4]:
df['title'] = df['title'].fillna('Unknown')

# Combine purpose and title into a new text column
df['text'] = df['purpose'].astype(str) + ' - ' + df['title'].astype(str)

# Drop the original text source columns
df.drop(['purpose', 'title'], axis=1, inplace=True)

print("Shape after creating text column:", df.shape)
df[['text']].head()


Shape after creating text column: (668640, 14)


,text
0,debt_consolidation - Debt consolidation
1,small_business - Business
2,home_improvement - Unknown
4,major_purchase - Major purchase
5,debt_consolidation - Debt consolidation


### Step 4: Save Cleaned Dataset

Now that we’ve filtered and cleaned the dataset, we save the result to a new file:  
`loan_data_cleaned.csv` — this will be used in the modeling steps.

In [5]:
output_path = '/content/drive/MyDrive/LoanApprovalProject/Data/loan_data_cleaned.csv'

df.to_csv(output_path, index=False)

print(" Saved cleaned dataset to:", output_path)


 Saved cleaned dataset to: /content/drive/MyDrive/LoanApprovalProject/Data/loan_data_cleaned.csv


### Step 5: Create a Balanced Sample (~20,000 rows)

To keep training efficient and balanced, we randomly sample 10,000 examples from each class.  
We save this as `loan_data_sampled.csv` for the upcoming training steps.

In [6]:
df_sampled = (
    df.groupby('loan_status')
      .apply(lambda x: x.sample(n=10000, random_state=42))
      .reset_index(drop=True)
)

sampled_path = '/content/drive/MyDrive/LoanApprovalProject/Data/loan_data_sampled.csv'
df_sampled.to_csv(sampled_path, index=False)

print(" Saved sampled dataset:", df_sampled.shape)
df_sampled['loan_status'].value_counts()


<ipython-input-6-9bf0f36c2137>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=10000, random_state=42))


 Saved sampled dataset: (20000, 14)


,count
loan_status,
0,10000
1,10000
